In [24]:
using DataFrames;
using MaxEntChemostat2018;
using MaxEntChemostat2018.FBA;
using Chemostat.
using Plots;
plotly();

## Tools

In [25]:
load_cho() = MaxEntChemostat2018.FBA.load_cho();


In [66]:
printlnall(collect(eachrow(rxns))[find((row) -> row[:an] > 0.5 || row[:ap] > 0.5, eachrow(rxns))])

DataFrameRow (row 586)
id  HMGLm
lb  0.0
ub  1000.0
ap  0.9113444152814232
an  0.9113444152814232

DataFrameRow (row 587)
id  HMGLx
lb  0.0
ub  1000.0
ap  0.9113444152814232
an  0.9113444152814232

DataFrameRow (row 708)
id  NMNS
lb  0.0
ub  1000.0
ap  15.188746438746438
an  15.188746438746438

DataFrameRow (row 710)
id  THD1m
lb  0.0
ub  1000.0
ap  0.9116809116809116
an  0.001073409642314236

DataFrameRow (row 749)
id  SERPT
lb  0.0
ub  1000.0
ap  11.396011396011396
an  11.396011396011396

DataFrameRow (row 775)
id  P45011A1m
lb  0.0
ub  1000.0
ap  2.427244793657007
an  2.427244793657007

DataFrameRow (row 776)
id  P45011B11m
lb  0.0
ub  1000.0
ap  2.4824032176973354
an  2.4824032176973354

DataFrameRow (row 777)
id  P45011B12m


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[66], in expression starting on line 1


lb  0.0
ub  1000.0
ap  12.634986437381649
an  12.634986437381649

DataFrameRow (row 778)
id  P45011B21m
lb  0.0
ub  1000.0
ap  2.4824032176973354
an  2.4824032176973354

DataFrameRow (row 784)
id  UGT1A1r
lb  0.0
ub  1000.0
ap  1.0981707833859422
an  1.0981707833859422

DataFrameRow (row 785)
id  UGT1A2r
lb  0.0
ub  1000.0
ap  1.0981707833859422
an  1.0981707833859422

DataFrameRow (row 786)
id  UGT1A4r
lb  0.0
ub  1000.0
ap  1.0981707833859422
an  1.0981707833859422

DataFrameRow (row 802)
id  RDH1
lb  -1000.0
ub  1000.0
ap  1.0485913263691042
an  1.687638493194049

DataFrameRow (row 803)
id  RDH1a
lb  -1000.0
ub  1000.0
ap  1.0485913263691042
an  1.687638493194049

DataFrameRow (row 804)
id  RDH2a
lb  -1000.0
ub  1000.0
ap  0.6291547958214625
an  1.012583095916429

DataFrameRow (row 805)
id  RDH3a
lb  -1000.0
ub  1000.0
ap  0.6291547958214625
an  1.012583095916429

DataFrameRow (row 808)
id  UGT1A5r2
lb  0.0
ub  1000.0
ap  1.0981707833859422
an  1.0981707833859422

DataFrameRow (row 

## Chemostat

In [65]:
findmax(rxns[:ap])

(1936.431623931624, 918)

In [26]:
function s(c, st, ξ)
    return max(c - st * ξ, 0);
end
function μ(z::Real, w::Real, τ::Real)
    return max(z - w * τ, 0);
end
function X(ξ::Real, μ::Real; ret_factor::Real = 1.0)
    return ξ * μ / ret_factor;
end
function transp_lac(fbares; lacindex = 271)#47
    return fbares.u[lacindex]; 
end
function transp_glc(fbares; glcindex = 247)#35
    return fbares.u[glcindex];
end

transp_glc (generic function with 1 method)

## Loading model

In [27]:
S, mets, rxns = load_cho();
println("S $(size(S)), mets $(size(mets)) rxns $(size(rxns))");

S (2054, 3578), mets (2054, 6) rxns (3578, 5)


## Max ξ

In [28]:
ξmax = MaxEntChemostat2018.FBA.find_max_xi(S, mets, rxns)*0.8;
println("ξmax $ξmax")

Academic license - for non-commercial use only
ξmax 392.2760009765625


## Mets of interest

In [29]:
lacindex = 217;
glcindex = 247;

## Testing FBA at ξmax

In [30]:
LP = MaxEntChemostat2018.FBA.gurobi_problem(S, mets, rxns);
fbares = MaxEntChemostat2018.FBA.model_solve!(LP, S, mets, rxns, ξmax);
if fbares != nothing
    println("ξmax $ξmax");
    println("μ $(fbares.μ)");
    println("Glc trans $(transp_glc(fbares))");
    println("Lac trans $(transp_lac(fbares))")
else
    println("NO RESULTS: fbares == nothing")
end

Academic license - for non-commercial use only
ξmax 392.2760009765625
μ 0.001379418066128686
Glc trans 0.06367404056791218
Lac trans 0.0


## Solving for all ξrange

In [31]:
ξrange = filter((ξ) -> ξ < ξmax, logrange(-3,log10(ξmax)+1; base = 1:0.5:9));
println("ξrange ($(first(ξrange)),$(last(ξrange))) length $(length(ξrange)) ");
@time fbaress = [begin
#             LP = MaxEntChemostat2018.FBA.gurobi_problem(S, mets, rxns);
            return fbares = MaxEntChemostat2018.FBA.model_solve!(LP, S, mets, rxns, ξ);
        end for ξ in ξrange];

ξrange (0.001,350.0) length 91 
 16.233964 seconds (122.42 k allocations: 93.021 MiB, 0.26% gc time)


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[31], in expression starting on line 240


## S vs ξ

In [32]:
glcC = mets[:c][glcindex];
glcSs = [s(glcC, transp_glc(fbares), fbares.ξ) for fbares in fbaress];
# glcSs = [fbares.s[glcindex] for fbares in fbaress]
plot((ξrange, glcSs), xaxis = :log, lw = 2)

## W vs ξ

In [33]:
lacSs = [s(0, transp_lac(fbares), fbares.ξ) for fbares in fbaress];
# lacSs = [fbares.s[lacindx] for fbares in fbaress]
plot((ξrange, lacSs), xaxis = :log, lw = 2)

## μ vs ξ

In [42]:
τ = 0.005;
μs = [μ(fbaress[i].μ, lacSs[i], τ) for i in 1:length(fbaress)];
# μs = [fbares.μ for fbares in fbaress]
plot((ξrange, μs), xaxis = :log, lw = 2)

## X vs ξ

In [43]:
Xs = [X(ξrange[i], μs[i]) for i in 1:length(fbaress)];
plot((ξrange, Xs), xaxis = :log, lw = 2)

## X vs D (τ > 0)

In [44]:
plot((μs, Xs) ,lw =2)

## X vs D (τ = 0)

In [45]:
τ = 0.0;
μsnt = [fbares.μ for fbares in fbaress];
Xsnt = [X(ξrange[i], μsnt[i]) for i in 1:length(fbaress)];
plot((μsnt, Xsnt) ,lw =2)